In [1]:
%config Completer.use_jedi = False
### reload magic
%reload_ext autoreload
%autoreload 2
import numpy as np
import os
from utility import load_data
import pickle
from new_fig import cal_interval_rmse, special_bar

/home/handsomedong/anaconda3/envs/dlra/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load data
path = os.path.join(os.getcwd(), "repro_data")
fileList =[os.path.join(os.getcwd(), "repro_data", i) for i in sorted(os.listdir(path))]
# model decision
fileList = [x for x in fileList if 'mt20_dt19' in x][0:5]
fileList

['/wk171/handsomedong/after_Meeting/evaluation/repro_data/BalGRUAdvPONIAtten_addponi_mt20_dt19_v15.pkl',
 '/wk171/handsomedong/after_Meeting/evaluation/repro_data/BalGRUAdvPONIAtten_addponi_mt20_dt19_v16.pkl',
 '/wk171/handsomedong/after_Meeting/evaluation/repro_data/BalGRUAdvPONIAtten_addponi_mt20_dt19_v17.pkl',
 '/wk171/handsomedong/after_Meeting/evaluation/repro_data/BalGRUAdvPONIAtten_addponi_mt20_dt19_v18.pkl',
 '/wk171/handsomedong/after_Meeting/evaluation/repro_data/BalGRUAdvPONIAtten_addponi_mt20_dt19_v19.pkl']

In [3]:
data_loader = load_data(*fileList, file_num = len(fileList))
all_data, target, datetime, = data_loader._return
all_data = np.concatenate(all_data, axis=1) # [B, N_exps, tar_t, H, W]
print(all_data.shape)

/wk171/handsomedong/after_Meeting/evaluation/repro_data/BalGRUAdvPONIAtten_addponi_mt20_dt19_v15.pkl has been loaded.
/wk171/handsomedong/after_Meeting/evaluation/repro_data/BalGRUAdvPONIAtten_addponi_mt20_dt19_v16.pkl has been loaded.
/wk171/handsomedong/after_Meeting/evaluation/repro_data/BalGRUAdvPONIAtten_addponi_mt20_dt19_v17.pkl has been loaded.
/wk171/handsomedong/after_Meeting/evaluation/repro_data/BalGRUAdvPONIAtten_addponi_mt20_dt19_v18.pkl has been loaded.
/wk171/handsomedong/after_Meeting/evaluation/repro_data/BalGRUAdvPONIAtten_addponi_mt20_dt19_v19.pkl has been loaded.
(13087, 5, 3, 120, 120)


### show STD

In [ ]:
# manually collect different models
tar_t = 0
rmse_one = []
# rmse_all = []
threshold = [1, 3, 5, 10, 15, 20, 30, 40]
for i in range(all_data.shape[1]):
    rmse_one.append(cal_interval_rmse(all_data[:, i, tar_t], target[:, tar_t], threshold))
rmse_all.append(np.stack(rmse_one, axis=0)) # [N_models][N_exps, N_thsh]

In [ ]:
special_bar(rmse_all).plot(['1~3', '3~5', '5~10', '10~15', '15~20', '20~30', '30~40', '>=40'], 
                           ['CPN', 'CAPN', 'PONI', 'PONI_Atten'])

### save average

In [4]:
with open(fileList[0].split('_v')[0]+"_avg.pkl", 'wb') as handle:
    pickle.dump([datetime, all_data.mean(axis=1), target], handle, protocol=pickle.HIGHEST_PROTOCOL)